In [17]:
import requests
from bs4 import BeautifulSoup
from rich import print
import pandas as pd
from rich.progress import track

## Top 250 Movies

In [2]:
URL = 'https://www.imdb.com'

In [3]:
def request_url(url:str)->BeautifulSoup:
    """
    Takes website URL and return soup object
    """
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html5lib")
    return soup

In [4]:
# Find the top 250 movie's link
top_250_page = request_url(URL+'/chart/top/')
movies_url = list()
table = top_250_page.find('tbody', class_='lister-list')
for i in table.find_all('td', class_='titleColumn'):
    movies_url.append(URL+i.find('a')['href'])

In [5]:
movie_page = request_url(movies_url[0])

In [6]:
def catch(func, handle):
    try:
        return func()
    except Exception as e:
        return handle()

In [31]:
df_list =list()
for index, movie_url in enumerate(track(movies_url)):
    try:
        movie_page = request_url(movie_url)
        # Get movie Title, year, classification and rating link.
        title = movie_page.find('h1', 'TitleHeader__TitleText-sc-1wu6n3d-0').text.strip()
        print(index, title)
        year_rat_time = list()
        year_rat_time_header = ['Year', 'Certification', 'Duration']
        dic = {title: catch(lambda : x.contents[0].text, handle=lambda : x.text) for x , title in zip(movie_page.find('div', 'TitleBlock__TitleMetaDataContainer-sc-1nlhx7j-2 hWHMKr').find('ul'), year_rat_time_header)}
        rating_url= URL+movie_page.find('a', 'ipc-button ipc-button--single-padding ipc-button--center-align-content ipc-button--default-height ipc-button--core-baseAlt ipc-button--theme-baseAlt ipc-button--on-textPrimary ipc-text-button RatingBarButtonBase__Button-sc-15v8ssr-2 jjcqHZ')['href']
        dic['Title']=title
        data = pd.DataFrame([dic])



        page = request_url(rating_url)



        table = page(text='Rating By Demographic')[0].parent.find_next_sibling().find_next_sibling()
        lst = list()
        header = ['All', '<18', '18-29', '30-44', '45+']
        for i in range(1,4):
            lst.append({j:x.text.strip() for x, j in zip(table.find_all('tr')[i].find_all('a'), header)})
        total = pd.DataFrame.from_dict(({'All': lst[0]}))
        male = pd.DataFrame({'Male': lst[1]})
        female = pd.DataFrame({'Female': lst[2]})
        total = helper(total)
        male = helper(male)
        female = helper(female)
        result = total.join([male, female])



        data.columns = pd.MultiIndex.from_tuples([('Year',''),('Certification',''),('Duration',''),('Title','')])

        df_list.append(pd.concat([data,result], axis=1))
    except Exception as e:

        print(index,e, movie_url)


Output()

0 The Shawshank Redemption

1 The Godfather

3 The Dark Knight

9 The Lord of the Rings: The Fellowship of the Ring

...

239 Before Sunset

242 La battaglia di Algeri

243 ديمن سلاير: موغين تراين - عرض خاص

248 Mandariinid

249 Ratsasan

In [32]:
len(df_list)

247

In [46]:
new_df = pd.concat(df_list).reset_index(drop=True)
new_df

Year Certification  Duration                     Title        All         \
                                                                   All    <18   
0    1994           R12  2h 22min  The Shawshank Redemption  2,472,416  1,430   
1    1972          PG12  2h 55min             The Godfather  1,708,395    980   
2    1974           R12  3h 22min    The Godfather: Part II  1,186,714    625   
3    2008          PG12  2h 32min           The Dark Knight  2,426,452  1,576   
4    1957      Approved  1h 36min              12 Angry Men    731,760    511   
..    ...           ...       ...                       ...        ...    ...   
242  1987            PG  1h 38min        The Princess Bride    409,214    147   
243  1984             R  2h 25min              Paris, Texas     99,486     67   
244  1957     Not Rated  1h 50min       Le notti di Cabiria     46,210     26   
245  2013     Not Rated  1h 27min               Mandariinid     44,112     35   
246  2018     Not Rated  2h 50min                  Ratsasan     35,169     75   

                                     Male                                    \
       18-29    30-44      45+        All    <18    18-29    30-44      45+   
0    422,839  950,198  238,058  1,438,110  1,020  334,004  789,779  192,715   
1    269,085  678,763  185,199  1,041,243    719  222,873  585,385  154,434   
2    179,237  476,375  138,373    739,410    461  152,632  415,560  116,435   
3    446,938  976,385  172,733  1,451,945  1,160  360,053  820,196  145,235   
4    131,183  275,942   78,325    438,896    389  107,422  235,505   65,419   
..       ...      ...      ...        ...    ...      ...      ...      ...   
242   50,136  166,072   70,686    213,997     98   31,461  117,947   53,800   
243   14,797   37,044   17,702     60,405     44   11,387   30,928   15,135   
244    5,332   18,551    8,869     27,940     20    4,240   15,532    7,175   
245    9,531   17,578    4,873     26,328     21    7,048   13,947    3,877   
246    7,049    5,744      775     15,572     67    6,554    5,365      582   

      Female                                
         All  <18   18-29    30-44     45+  
0    286,036  225  76,534  145,904  40,194  
1    160,147  159  38,443   83,251  26,807  
2    102,161  117  22,048   53,779  19,169  
3    259,931  216  74,282  142,027  23,924  
4     74,160   80  20,692   36,984  11,478  
..       ...  ...     ...      ...     ...  
242   84,437   36  17,682   46,083  15,763  
243   11,628   18   3,065    5,667   2,276  
244    5,642    6   1,002    2,858   1,574  
245    7,096   10   2,353    3,405     918  
246      926    7     261      303     175  

[247 rows x 19 columns]

In [37]:
new_df.to_csv('temp.csv')

In [26]:
movie_page = request_url('https://www.imdb.com/title/tt4154796/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=VEJVCG6NQCDBXFGYE3NN&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_76')
# Get movie Title, year, classification and rating link.
title = movie_page.find('h1', 'TitleHeader__TitleText-sc-1wu6n3d-0').text.strip()
print(index, title)
year_rat_time = list()
year_rat_time_header = ['Year', 'Certification', 'Duration']
dic = {title: catch(lambda : x.contents[0].text, handle=lambda : x.text) for x , title in zip(movie_page.find('div', 'TitleBlock__TitleMetaDataContainer-sc-1nlhx7j-2 hWHMKr').find('ul'), year_rat_time_header)}
rating_url= URL+movie_page.find('a', 'ipc-button ipc-button--single-padding ipc-button--center-align-content ipc-button--default-height ipc-button--core-baseAlt ipc-button--theme-baseAlt ipc-button--on-textPrimary ipc-text-button RatingBarButtonBase__Button-sc-15v8ssr-2 jjcqHZ')['href']
dic['Title']=title
data = pd.DataFrame([dic])

print()

page = request_url(rating_url)



table = page(text='Rating By Demographic')[0].parent.find_next_sibling().find_next_sibling()
lst = list()
header = ['All', '<18', '18-29', '30-44', '45+']
for i in range(1,4):
    lst.append({j:x.text.strip() for x, j in zip(table.find_all('tr')[i].find_all('a'), header)})
total = pd.DataFrame.from_dict(({'All': lst[0]}))
male = pd.DataFrame({'Male': lst[1]})
female = pd.DataFrame({'Female': lst[2]})
total = helper(total)
male = helper(male)
female = helper(female)
result = total.join([male, female])



data.columns = pd.MultiIndex.from_tuples([('Year',''),('Certification',''),('Duration',''),('Title','')])

df_list.append(pd.concat([data,result], axis=1))


76 Avengers: Endgame

In [28]:
df_list[-1]

Year Certification Duration              Title      All                  \
                                                       All    <18    18-29   
0  2019         PG-13  3h 1min  Avengers: Endgame  946,224  2,165  178,565   

                       Male                                   Female       \
     30-44     45+      All    <18    18-29    30-44     45+     All  <18   
0  243,073  62,167  451,559  1,374  135,607  199,738  50,448  80,892  380   

                          
    18-29   30-44    45+  
0  26,805  33,696  9,143

The difference in the counts is because only some IMDb users have chosen to provide age, gender and country information when registering for IMDb. Since this information is optional, we cannot assign every vote to every category.



In [9]:
def helper(df:pd.DataFrame()):
    header = ['All', '<18', '18-29', '30-44', '45+']
    df = df.T[header]
    df.columns = pd.MultiIndex.from_product([df.index,df.columns])
    return df.reset_index(drop=True)

In [12]:
data

,Year,Certification,Duration,Title
,,,,
0,1994,R12,2h 22min,The Shawshank Redemption


In [13]:
w

Year Certification  Duration                     Title        All         \
                                                                 All    <18   
0  1994           R12  2h 22min  The Shawshank Redemption  2,472,392  1,430   

                                   Male                                    \
     18-29    30-44      45+        All    <18    18-29    30-44      45+   
0  422,835  950,193  238,057  1,438,102  1,020  334,000  789,777  192,714   

    Female                                
       All  <18   18-29    30-44     45+  
0  286,033  225  76,534  145,901  40,194